In [2]:
import pandas
import sqlite3
import numpy
import statsmodels.formula.api as smf

In [3]:
conn = sqlite3.connect('/home/computerowner/Documents/PythonCodes/Bets/BettingData.db')
dfSeason = pandas.read_sql_query('SELECT * FROM SeasonResults', conn)
dfGame = pandas.read_sql_query('SELECT * FROM GameResults', conn)
dfSplits = pandas.read_sql_query('SELECT * FROM TeamSplits', conn)

conn.close()

In [4]:
dfSplits1 = dfSplits[['SchoolName','Side','Year','Value','GamesPlayed','TotalYds','TotalPlays']].copy()
dfSplits1 = dfSplits1[dfSplits1['Side'] == 'Offense']

wins = dfSplits1[(dfSplits1['Value'] == 'Win')]
wins.head()
losses = dfSplits1[(dfSplits1['Value'] == 'Loss')]
losses.head()

,SchoolName,Side,Year,Value,GamesPlayed,TotalYds,TotalPlays
3,air-force,Offense,2006,Loss,8,323.6,64.5
12,air-force,Offense,2007,Loss,4,363.0,69.0
22,air-force,Offense,2008,Loss,5,307.2,67.0
32,air-force,Offense,2009,Loss,5,306.2,75.2
42,air-force,Offense,2010,Loss,4,396.8,67.8


In [5]:
dfRecord = pandas.merge(wins, losses, how='inner', on = ['SchoolName','Year'])

dfRecord = dfRecord[['SchoolName','Year','GamesPlayed_x','GamesPlayed_y','TotalPlays_x','TotalPlays_y','TotalYds_x','TotalYds_y']]
dfRecord = dfRecord.rename( columns = {'GamesPlayed_x':'Wins','GamesPlayed_y':'Losses'})
dfRecord = dfRecord.drop_duplicates()

dfRecord['TotalYds'] = dfRecord['TotalYds_x'] + dfRecord['TotalYds_y']
dfRecord['TotalPlays'] = dfRecord['TotalPlays_x'] + dfRecord['TotalPlays_y']
dfRecord['YdsPerPlay'] = dfRecord['TotalYds']/dfRecord['TotalPlays']
dfRecord['TotalGames'] = dfRecord['Wins'] + dfRecord['Losses']
dfRecord['WinningPercentage'] = dfRecord['Wins']/dfRecord['TotalGames']
dfRecord = dfRecord[['SchoolName','Year','Wins','Losses','TotalPlays','YdsPerPlay','TotalGames','WinningPercentage']]
dfRecord.head()

,SchoolName,Year,Wins,Losses,TotalPlays,YdsPerPlay,TotalGames,WinningPercentage
0,air-force,2006,4,8,137.0,4.878832,12,0.333333
1,air-force,2007,9,4,142.2,5.677918,13,0.692308
2,air-force,2008,8,5,139.5,4.892473,13,0.615385
3,air-force,2009,8,5,149.2,4.819705,13,0.615385
4,air-force,2010,9,4,139.5,5.989964,13,0.692308


In [6]:
dfRecord.corr()

,Year,Wins,Losses,TotalPlays,YdsPerPlay,TotalGames,WinningPercentage
Year,1.000000,-0.004845,-0.016644,0.321986,0.177416,-0.073623,0.002142
Wins,-0.004845,1.000000,-0.978664,0.064030,0.294744,0.739717,0.993284
Losses,-0.016644,-0.978664,1.000000,-0.074844,-0.299516,-0.585673,-0.992722
TotalPlays,0.321986,0.064030,-0.074844,1.000000,0.218536,0.007472,0.069199
YdsPerPlay,0.177416,0.294744,-0.299516,0.218536,1.000000,0.181804,0.299690
TotalGames,-0.073623,0.739717,-0.585673,0.007472,0.181804,1.000000,0.667184
WinningPercentage,0.002142,0.993284,-0.992722,0.069199,0.299690,0.667184,1.000000


In [7]:
model = smf.ols('WinningPercentage ~ YdsPerPlay', data = dfRecord).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      WinningPercentage   R-squared:                       0.090
Model:                            OLS   Adj. R-squared:                  0.089
Method:                 Least Squares   F-statistic:                     117.5
Date:                Wed, 22 Jun 2016   Prob (F-statistic):           3.51e-26
Time:                        09:10:44   Log-Likelihood:                 187.72
No. Observations:                1193   AIC:                            -371.4
Df Residuals:                    1191   BIC:                            -361.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     -0.0630      0.054     -1.168      0.243        -0.169     0.043
YdsPerPlay     0.1047      0.010     10.841      0.000         0.086     0.124
==============================================================================
Omnibus:                       37.543   Durbin-Watson:                   1.154
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               21.185
Skew:                          -0.155   Prob(JB):                     2.51e-05
Kurtosis:                       2.425   Cond. No.                         51.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [8]:
model.params

Intercept    -0.062991
YdsPerPlay    0.104675
dtype: float64

In [10]:
dfRecord.columns

Index(['SchoolName', 'Year', 'Wins', 'Losses', 'TotalPlays', 'YdsPerPlay',
       'TotalGames', 'WinningPercentage'],
      dtype='object')